In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [3]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1411


In [4]:
def get_metrics(y, p_y):
    metrics = {}
    try:
        m = tf.keras.metrics.Accuracy(name="accuracy", dtype=None)
        m.update_state(y, p_y)
        metrics['accuracy'] = m.result().numpy()
    except:
        pass
    try:
        metrics['mse'] = mse(y,p_y)
    except:
        pass
    return metrics
    

# Handmade NN

In [5]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [6]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [7]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
# model_conv.compile(optimizer='adam', loss='mse')

In [8]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
34/34 [==============================] - 2s 27ms/step - loss: 2212785.7500
Epoch 2/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2198
Epoch 3/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2101
Epoch 4/30
34/34 [==============================] - 1s 28ms/step - loss: 1203.2092
Epoch 5/30
34/34 [==============================] - 1s 29ms/step - loss: 1203.2091
Epoch 6/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2086
Epoch 7/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2086
Epoch 8/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2078
Epoch 9/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2078
Epoch 10/30
34/34 [==============================] - 1s 27ms/step - loss: 1203.2074
Epoch 11/30
34/34 [==============================] - 1s 28ms/step - loss: 1203.2069
Epoch 12/30
34/34 [==============================] - 1s 29ms/step - loss: 1203.206

In [9]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=30)

Epoch 1/30
34/34 [==============================] - 3s 14ms/step - loss: 1203.1193
Epoch 2/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.9297
Epoch 3/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.8412
Epoch 4/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.7147
Epoch 5/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.5892
Epoch 6/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.5847
Epoch 7/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.5077
Epoch 8/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.4453
Epoch 9/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.3577
Epoch 10/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.2385
Epoch 11/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.1178
Epoch 12/30
34/34 [==============================] - 0s 14ms/step - loss: 1202.0139
E

In [10]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
print(get_metrics(Y_predicted, Y_test_flat))
print(get_metrics(Y_ann_predicted, Y_test_flat))

12/12 [==============================] - 0s 3ms/step
{'accuracy': 0.0, 'mse': 154.74663666036696}
{'accuracy': 0.0, 'mse': 156.4696412212042}


In [11]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
# Y_predicted

In [12]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
# Y_ann_predicted

# AutoKeras

In [13]:
import autokeras as ak
import numpy as np

In [14]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=500
)
reg.fit(train_set,epochs=15, validation_split=0.1)

# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|

Trial 16 Complete [00h 00m 10s]
val_loss: 3.7636752128601074

Best val_loss So Far: 0.08541024476289749
Total elapsed time: 00h 02m 32s

Search: Running Trial #17

Value             |Best Value So Far |Hyperparameter
False             |False             |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
512               |32                |structured_data_block_1/dense_block_1/units_0
0.25              |0.25              |structured_data_block_1/dense_block_1/dropout
1024              |1024              |structured_data_block_1/dense_block_1/units_1
0                 |0                 |regression_head_1/dropout
adam              |adam              |optimizer
0.01              |0.01              |learning_rate

Epoch 1/15
31/31 [==============================] - 3s 34ms/step - loss: 2176656896.0000 - mean_squared_error: 217665

In [ ]:
predicted_y = reg.predict(test_set)
print(reg.evaluate(test_set))
print(get_metrics(Y_test_flat, predicted_y))
predicted_y

In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True)
regressor.fit(train_set, epochs=3, validation_split=0.15)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))

In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

# Autogoal